In [ ]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

runtime = "automatic"
# load flow
flow = "../contoso-chat_hf" #"../contoso-chat"


eval_flow = "multi_flow/"   #"multi_flow_hf/"
data = "../data/salestestdata.jsonl"


In [ ]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output_hf = pf_client.test(
    flow=flow, # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output_hf["answer"] = "".join(list(output_hf["answer"]))

In [ ]:
output_hf

Test the groundedness of the prompt flow with the answer from the above question.

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [ ]:
print("question",question)
print("context",output_hf["context"])
print("answer", output_hf["answer"])
test_multi_hf = pf_client.test(
    eval_flow,
    inputs={
        "question": question,
        "context": str(output_hf["context"]),
        "answer": output_hf["answer"],
    },
)

In [ ]:
test_multi_hf

In [ ]:

# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name_hf = timestamp+"chat_base_run"
print(run_name_hf)

# create base run in Azure Ai Studio
local_base_run_hf = pf_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name_hf,
    name=run_name_hf
)
print(local_base_run_hf)

In [ ]:
pf_client.stream(local_base_run_hf)
details = pf_client.get_details(local_base_run_hf)
details.head(10)

In [ ]:

run_name = timestamp+"chat_eval_run"
print(run_name)

run_name_hf = timestamp+"chat_eval_run"
print(run_name_hf)

local_eval_run_variant = pf_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=local_base_run_hf,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

In [ ]:

metrics_hf = pf_client.get_metrics(local_eval_run_variant)
import json
print(json.dumps(metrics_hf, indent=4))
pf_client.stream(local_eval_run_variant)

In [ ]:
pf_client.visualize([local_base_run_hf, local_eval_run_variant])